# Minimizing Row Size of Database
- Heroku PostgreSQL has a limit of 1GB and 10000 rows.
- Convert 610k+ rows of baby names to 1842 rows of distinct baby names.
- Name entries will be stored in Postgres database as JSON data types.

In [1]:
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from config import DATABASE_URI
from models import NameEntry

import us

In [2]:
# Create PostgreSQL database connection with SQLAlchemy
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)

In [3]:
def get_totals(name):
    '''
    Function to retrieve name entries.
    '''
    
    s = Session()
    q = s.query(NameEntry).\
    filter(NameEntry.name == name).\
    all()
    
    s.close()

    return [
        {'state': v.state,
         'year': v.year,
         'name': v.name,
         'gender': v.gender,
         'births': v.births}
        for v in q
    ]

In [4]:
# Sample
sample = get_totals('Logan')
print('Number of Entries:', len(sample))
print(sample[0])

Number of Entries: 1543
{'state': 'Alabama', 'year': 2019, 'name': 'Logan', 'gender': 'Male', 'births': 129}


## JSON Format
The JSON format of the data will be as follows:
```
{
  "Female": {
    "Alabama": {
      "1960": <births>
      ...
    },
    ...
  },
  "Male": ...
}
```

In [5]:
def to_json(name_data):
    '''
    Function to convert name entries into a single JSON by name
    '''
    
    states = [ state.name for state in us.states.STATES ] + ['District of Columbia']
    json_data = {
        "Female": { f"{state}": {} for state in states },
        "Male": { f"{state}": {} for state in states }
    }

    for entry in name_data:
        json_data[entry['gender']][entry['state']][f"{entry['year']}"] = entry['births']
        
    return json_data

In [6]:
# Sample
sample_json = to_json(sample)
sample_json['Female']['Hawaii']

{'2008': 12, '2004': 15, '2003': 11}